In [1]:
import os
import sys
import numpy as np
import random
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Conv1D, MaxPooling1D, Embedding, Flatten, GlobalAveragePooling1D, Dense, Input, GlobalMaxPooling1D, Concatenate, Dropout
from keras.models import Sequential, Model
from keras.optimizers import RMSprop, Adam
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
import pandas as pd
import re
from sklearn.model_selection import train_test_split



[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# with open('/content/drive/MyDrive/glove/Glove/glove.6B.300d.txt', 'r') as f:
#     gloveData = f.read()

# print(gloveData)

In [43]:
data = pd.read_csv('/content/drive/MyDrive/glove/file.csv')


In [44]:
data.rename( columns={'Unnamed: 0':'id'}, inplace=True )

In [45]:
data.head()

,id,tweets,labels
0,0,ChatGPT: Optimizing Language Models for Dialog...,neutral
1,1,"Try talking with ChatGPT, our new AI system wh...",good
2,2,ChatGPT: Optimizing Language Models for Dialog...,neutral
3,3,"THRILLED to share that ChatGPT, our new model ...",good
4,4,"As of 2 minutes ago, @OpenAI released their ne...",bad


In [46]:
my_dict = {data['id'][i]: [data['tweets'][i], data['labels'][i]] for i in range(len(data['id']))}

print(my_dict)

# for key in my_dict:
#     print(my_dict[key][1]) #label
#     print(my_dict[key][0]) #word

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [47]:
#mapping the labels of the tweets to 0, 1, 2 instead of bad, neutral, good
mapping = {'bad': 0, 'neutral': 1, 'good': 2}
my_dict = {key: [value[0], mapping[value[1]]] for key, value in my_dict.items()}

print(my_dict)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [48]:
# texts = []
# stop_words = set(stopwords.words('english'))


# for key in my_dict.keys():
#   value = my_dict[key][0]

#   # remove the URLs from the tweet body
#   url_regex = re.compile(r'https?://\S+')
#   my_dict[key][0] = url_regex.sub('', value)

#   # remove the english symbols and \n from the tweet body
#   replaces = my_dict[key][0].replace("\/", "")
#   replaces = replaces.replace("\\n", " ")
#   my_dict[key][0] = re.sub('[\\W]', ' ',replaces)

#   # remove the stop words from the tweet body
#   words = my_dict[key][0].split()
#   filtered_words = [word.lower() for word in words if word.isalpha() and word.lower() not in stop_words]
#   my_dict[key][0] = ' '.join(filtered_words)

#   texts.append(my_dict[key][0])

# print(my_dict)

stop_words = set(stopwords.words('english'))
def preprocess(value):
# remove the URLs from the tweet body
  url_regex = re.compile(r'https?://\S+')
  value = url_regex.sub('', value)

  # remove the english symbols and \n from the tweet body
  replaces = value.replace("/", "")
  replaces = replaces.replace("\n", " ")
  value = re.sub('[\W]', ' ',replaces)

  # remove the stop words from the tweet body
  words = value.split()
  filtered_words = [word.lower() for word in words if word.isalpha() and word.lower() not in stop_words]
  value = ' '.join(filtered_words)
  return value

In [49]:
texts = []

for key in my_dict.keys():
  my_dict[key][0] = preprocess(my_dict[key][0])
  texts.append(my_dict[key][0])

print(my_dict)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [50]:
lemmatizer = WordNetLemmatizer()

lemmated_tokens = []

for key in texts:
  lemmated_string = ""
  words = key.split()
  for word_2 in words:
    lemmated_string = lemmated_string + lemmatizer.lemmatize(word_2)+ " "
  lemmated_tokens.append(lemmated_string)
print(lemmated_tokens[0])

chatgpt optimizing language model dialogue openai 


In [61]:
MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 40000
def tokenize(tokens):
    tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
    tokenizer.fit_on_texts(tokens)
    sequences = tokenizer.texts_to_sequences(tokens)
    word_index = tokenizer.word_index # the dictionary
    # print('Found %s unique tokens.' % len(word_index)) #only top MAX_NUM_WORDS will be used to generate the sequences
    data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
    # print('Shape of samples:', data.shape)
    # print('Sampele:(the zeros at the begining are for padding text to max length)')
    # print(data[2])
    return word_index,data

In [62]:
word_index ,data = tokenize(lemmated_tokens)
print(word_index)

{'chatgpt': 1, 'n': 2, 'ai': 3, 'openai': 4, 'like': 5, 'write': 6, 'google': 7, 'new': 8, 'asked': 9, 'using': 10, 'use': 11, 'answer': 12, 'question': 13, 'code': 14, 'time': 15, 'chatbot': 16, 'make': 17, 'get': 18, 'know': 19, 'good': 20, 'one': 21, 'thing': 22, 'think': 23, 'human': 24, 'people': 25, 'see': 26, 'via': 27, 'amp': 28, 'search': 29, 'way': 30, 'work': 31, 'could': 32, 'tool': 33, 'would': 34, 'need': 35, 'day': 36, 'model': 37, 'future': 38, 'writing': 39, 'year': 40, 'world': 41, 'going': 42, 'ni': 43, 'bot': 44, 'gpt': 45, 'ask': 46, 'language': 47, 'really': 48, 'better': 49, 'even': 50, 'prompt': 51, 'job': 52, 'technology': 53, 'say': 54, 'used': 55, 'help': 56, 'first': 57, 'tech': 58, 'story': 59, 'chat': 60, 'content': 61, 'u': 62, 'artificialintelligence': 63, 'much': 64, 'take': 65, 'intelligence': 66, 'twitter': 67, 'text': 68, 'still': 69, 'give': 70, 'got': 71, 'essay': 72, 'response': 73, 'create': 74, 'article': 75, 'well': 76, 'right': 77, 'also': 78,

In [63]:
VALIDATION_SPLIT = 0.2

# X = [my_dict[key][0] for key in my_dict]
y = [my_dict[key][1] for key in my_dict]

temp =[]
for i in y:
  if i == 0:
    temp_1=[1,0,0]
  elif i==1:
    temp_1 = [0,1,0]
  else:
      temp_1=[0,0,1]
  temp.append(temp_1)

X_train, X_test, y_train, y_test = train_test_split(data, temp, test_size=0.2, random_state=42)
X_train, x_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# print(X_train)
# print(X_test)
# print(y_train)
# print(y_test)

X_train_array = np.array(X_train)
X_test_array = np.array(X_test)
y_train_array = np.array(y_train)
y_test_array = np.array(y_test)
x_val_array = np.array(x_val)
y_val_array = np.array(y_val)


In [54]:
EMBEDDING_DIM = 300
embeddings_index = {}
with open('/content/drive/MyDrive/glove/Glove/glove.6B.300d.txt', 'r') as f:
  for line in f:
    values = line.split(sep=' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [64]:
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))#+1 to include the zerors vector for non-existing words
for word, i in word_index.items():
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    # words not found in embedding index will be all-zeros.
    embedding_matrix[i] = embedding_vector
print ('Shape of Embedding Matrix: ',embedding_matrix.shape)

Shape of Embedding Matrix:  (88595, 300)


In [65]:
embedding_layer = Embedding(len(word_index) + 1, #vocab size
    EMBEDDING_DIM, #embedding vector size
    weights=[embedding_matrix], #weights matrix
    input_length=MAX_SEQUENCE_LENGTH, #padded sequence length
    trainable=False)

In [70]:
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH, ), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

filter_sizes = [3, 3, 4]

conv_layers = []
for filter_size in filter_sizes:
    conv_layer = Conv1D(128, kernel_size=filter_size, activation='relu')(embedded_sequences)
    pool_layer = GlobalMaxPooling1D()(conv_layer)
    conv_layers.append(pool_layer)

merged_layer = Concatenate()(conv_layers)
flatten_layer = Flatten()(merged_layer)
dropout_layer = Dropout(0.5)(flatten_layer)

dense_layer1 = Dense(128, activation='relu')(dropout_layer)
output_layer = Dense(3, activation='softmax')(dense_layer1)
# output_layer = Dense(3, activation='softmax')(merged_layer)
model = Model(inputs=sequence_input, outputs=output_layer)

In [ ]:
# sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH, ), dtype='int32')
# embedded_sequences = embedding_layer(sequence_input)
# x = Conv1D(128, 5, activation='relu')(embedded_sequences)
# x = MaxPooling1D(5)(x)
# x = Conv1D(64, 5, activation='relu')(x)
# x = MaxPooling1D(35)(x)
# x = Flatten()(x)
# x = Dense(128, activation='relu')(x)
# preds = Dense(3, activation='softmax')(x)

In [71]:
model = Model(sequence_input, output_layer)
model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['acc'])
model.summary()

Model: "model_12"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 1000)]       0           []                               
                                                                                                  
 embedding_3 (Embedding)        (None, 1000, 300)    26578500    ['input_7[0][0]']                
                                                                                                  
 conv1d_14 (Conv1D)             (None, 998, 128)     115328      ['embedding_3[1][0]']            
                                                                                                  
 conv1d_15 (Conv1D)             (None, 998, 128)     115328      ['embedding_3[1][0]']            
                                                                                           

In [72]:
model.fit(X_train_array, y_train_array, validation_data=(x_val_array, y_val_array),epochs=20, batch_size=128)

Epoch 1/20
1097/1097 [==============================] - 107s 96ms/step - loss: 0.6889 - acc: 0.6982 - val_loss: 0.5717 - val_acc: 0.7647
Epoch 2/20
1097/1097 [==============================] - 104s 95ms/step - loss: 0.5231 - acc: 0.7811 - val_loss: 0.4796 - val_acc: 0.8081
Epoch 3/20
1097/1097 [==============================] - 105s 96ms/step - loss: 0.4699 - acc: 0.8062 - val_loss: 0.4550 - val_acc: 0.8177
Epoch 4/20
1097/1097 [==============================] - 105s 96ms/step - loss: 0.4336 - acc: 0.8222 - val_loss: 0.4219 - val_acc: 0.8369
Epoch 5/20
1097/1097 [==============================] - 105s 96ms/step - loss: 0.4072 - acc: 0.8360 - val_loss: 0.4154 - val_acc: 0.8356
Epoch 6/20
1097/1097 [==============================] - 104s 95ms/step - loss: 0.3827 - acc: 0.8466 - val_loss: 0.4047 - val_acc: 0.8436
Epoch 7/20
1097/1097 [==============================] - 104s 94ms/step - loss: 0.3659 - acc: 0.8553 - val_loss: 0.4193 - val_acc: 0.8346
Epoch 8/20
1097/1097 [===================

In [73]:
print('Acuracy on testing set:')
model.evaluate(X_test_array, y_test_array)

Acuracy on testing set:
1371/1371 [==============================] - 17s 12ms/step - loss: 0.4140 - acc: 0.8500


[0.4139508903026581, 0.8500421643257141]

In [74]:
model.predict(X_test_array)

1371/1371 [==============================] - 13s 9ms/step


array([[9.9869257e-01, 1.2407972e-03, 6.6704510e-05],
       [9.9772769e-01, 1.9735706e-03, 2.9874549e-04],
       [7.6850468e-01, 1.9749470e-01, 3.4000617e-02],
       ...,
       [4.2080781e-03, 8.4827608e-01, 1.4751585e-01],
       [9.9205208e-01, 4.3469789e-03, 3.6009965e-03],
       [4.9207555e-03, 7.8182004e-02, 9.1689730e-01]], dtype=float32)

In [106]:
tweet = input("Enter your tweet: ")
# {'bad': 0, 'neutral': 1, 'good': 2}
# tweet = "I had been seeing lots of tweets about ChatGPT so I signed up for it last night.  Turn out it works decently well for generating stories to read to your kids based on what they want to hear a story about."
tweet = preprocess(tweet)
# print(tweet)

lemmatize_token = []
words = tweet.split()
lemmatize_string = ""
for word_2 in words:
  lemmatize_string = lemmatize_string + lemmatizer.lemmatize(word_2)+" "
tweet = lemmatize_string
lemmatize_token.append(tweet)
# print(tweet)

tweet_word_index, tweet_data=tokenize(lemmatize_token)

# print(tweet_word_index)
# print(tweet_data)

label_vec = model.predict(tweet_data[0].reshape(1,-1))

actual_labels = {'negative': 0, 'neutral': 1, 'positive': 2}
probs = np.array(label_vec)
index = np.argmax(probs)
output_label = list(actual_labels.keys())[list(actual_labels.values()).index(index)]


print(output_label)

# print(label_vec)

Enter your tweet: negative
1/1 [==============================] - 0s 18ms/step
negative
